### Loading Data

In [ ]:
import pandas as pd

bank_df = pd.read_csv('bank.csv', delimiter=';')

file_path = 'adult.data'
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
    'hours-per-week', 'native-country', 'income'
]
adult_df = pd.read_csv(file_path, names=columns, na_values="?", skipinitialspace=True)



### Binning Columns

In [ ]:
# Binary indicators for capital gains and losses
adult_df['positive_capital_gain'] = (adult_df['capital-gain'] > 0).astype(int)
adult_df['positive_capital_loss'] = (adult_df['capital-loss'] > 0).astype(int)

# Interaction feature between age and education-num
adult_df['age_education_interaction'] = adult_df['age'] * adult_df['education-num']

# Aggregating less common categories into 'Other'
top_occupations = adult_df['occupation'].value_counts().nlargest(5).index
adult_df['occupation_aggregated'] = adult_df['occupation'].apply(lambda x: x if x in top_occupations else 'Other')

top_countries = adult_df['native-country'].value_counts().nlargest(5).index
adult_df['native_country_aggregated'] = adult_df['native-country'].apply(lambda x: x if x in top_countries else 'Other')

# Binning age and hours-per-week
age_bins = [0, 25, 35, 45, 55, 65, 100]
age_labels = ['0-25', '26-35', '36-45', '46-55', '56-65', '66+']
adult_df['age_binned'] = pd.cut(adult_df['age'], bins=age_bins, labels=age_labels, right=False)

hours_bins = [0, 20, 30, 40, 50, 100]
hours_labels = ['0-20', '21-30', '31-40', '41-50', '51+']
adult_df['hours_per_week_binned'] = pd.cut(adult_df['hours-per-week'], bins=hours_bins, labels=hours_labels, right=False)


adult_df.columns


### Pipeline

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

### Preprocessing Function ###
def preprocess(df):
    df = df.dropna()
    cat_cols = df.select_dtypes(include=['object', 'category']).columns
    df = pd.get_dummies(df, columns=cat_cols)

    scaler = StandardScaler()
    num_cols = df.select_dtypes(include=['int64', 'float64']).columns
    df[num_cols] = scaler.fit_transform(df[num_cols])
    return df


In [ ]:
### Transaction Conversion Function ###
def dataframe_to_transactions(df):
    transactions = []
    for _, row in df.iterrows():
        transaction = []
        for col in df.columns:
            # This check ensures that only non-zero entries (True, for one-hot encoded columns) are included.
            if row[col] == 1:
                transaction.append(col)
        transactions.append(transaction)
    return transactions


In [21]:
def estimate_dbscan_params(df, eps_values, min_samples_values):
    best_silhouette = -1
    best_params = None
    best_labels = None
    
    for eps in eps_values:
        for min_samples in min_samples_values:
            dbscan = DBSCAN(eps=eps, min_samples=min_samples)
            labels = dbscan.fit_predict(df)
            
            if len(set(labels)) < 2 or np.count_nonzero(labels == -1) > 0.5 * len(labels):
                continue
            
            silhouette = silhouette_score(df[labels != -1], labels[labels != -1]) if -1 in labels else silhouette_score(df, labels)
            
            print(f"Testing eps={eps}, min_samples={min_samples} --> Silhouette: {silhouette:.4f}, clusters: {len(set(labels))}")

            if silhouette > best_silhouette:
                best_silhouette = silhouette
                best_params = {'eps': eps, 'min_samples': min_samples}
                best_labels = labels
                
    print(f"Best Parameters: eps={best_params['eps']}, min_samples={best_params['min_samples']} with silhouette={best_silhouette:.4f}")
    return best_params, best_labels

preprocessed_df = preprocess(bank_df)
eps_values=np.linspace(0.1, 2.0, 20)
min_samples_values=range(2, 20)

print(estimate_dbscan_params(preprocessed_df, eps_values, min_samples_values))

Testing eps=1.5999999999999999, min_samples=2 --> Silhouette: -0.1999, clusters: 161
Testing eps=1.5999999999999999, min_samples=3 --> Silhouette: -0.1791, clusters: 50
Testing eps=1.7, min_samples=2 --> Silhouette: -0.2021, clusters: 133
Testing eps=1.7, min_samples=3 --> Silhouette: -0.1359, clusters: 42
Testing eps=1.7, min_samples=4 --> Silhouette: -0.1015, clusters: 19
Testing eps=1.7, min_samples=5 --> Silhouette: -0.0585, clusters: 12
Testing eps=1.8, min_samples=2 --> Silhouette: -0.2048, clusters: 121
Testing eps=1.8, min_samples=3 --> Silhouette: -0.1281, clusters: 36
Testing eps=1.8, min_samples=4 --> Silhouette: -0.1058, clusters: 22
Testing eps=1.8, min_samples=5 --> Silhouette: -0.0266, clusters: 16
Testing eps=1.8, min_samples=6 --> Silhouette: 0.0852, clusters: 7
Testing eps=1.8, min_samples=7 --> Silhouette: 0.2585, clusters: 4
Testing eps=1.8, min_samples=8 --> Silhouette: 0.0932, clusters: 4
Testing eps=1.8, min_samples=9 --> Silhouette: 0.3091, clusters: 3
Testing e

In [ ]:
### FP-Growth Association Rule Mining Function ###
def apply_fp_growth(transactions, min_support=0.1, min_confidence=0.3, min_lift=3):
    te = TransactionEncoder()
    te_ary = te.fit(transactions).transform(transactions)
    df = pd.DataFrame(te_ary, columns=te.columns_)
    
    frequent_itemsets = fpgrowth(df, min_support=min_support, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)
    rules = rules[rules['lift'] >= min_lift]
    
    return rules



In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

### Preprocessing Function ###
def preprocess(df):
    df_clean = df.dropna()
    cat_cols = df_clean.select_dtypes(include=['object', 'category']).columns
    df_clean = pd.get_dummies(df_clean, columns=cat_cols)
    scaler = StandardScaler()
    num_cols = df_clean.select_dtypes(include=['int64', 'float64']).columns
    df_clean[num_cols] = scaler.fit_transform(df_clean[num_cols])
    return df_clean

### Transaction Conversion Function ###
def dataframe_to_transactions(df):
    transactions = []
    for _, row in df.iterrows():
        transaction = [col for col in df.columns if row[col] == 1]
        transactions.append(transaction)
    return transactions

### DBSCAN with Parameter Estimation Function ###
def estimate_dbscan_params(df, eps_values, min_samples_values):
    best_silhouette = -1
    best_params = None
    best_labels = None
    
    for eps in eps_values:
        for min_samples in min_samples_values:
            dbscan = DBSCAN(eps=eps, min_samples=min_samples)
            labels = dbscan.fit_predict(df)
            if len(set(labels)) < 3 or np.count_nonzero(labels == -1) > 0.5 * len(labels):
                continue
            if -1 in labels:
                silhouette = silhouette_score(df[labels != -1], labels[labels != -1])
            else:
                silhouette = silhouette_score(df, labels)
            if silhouette > best_silhouette:
                best_silhouette = silhouette
                best_params = {'eps': eps, 'min_samples': min_samples}
                best_labels = labels

    return best_params, best_labels

### FP-Growth Association Rule Mining Function ###
def apply_fp_growth(transactions, min_support=0.1, min_confidence=0.3, min_lift=3):
    te = TransactionEncoder()
    te_ary = te.fit(transactions).transform(transactions)
    df = pd.DataFrame(te_ary, columns=te.columns_)
    frequent_itemsets = fpgrowth(df, min_support=min_support, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)
    rules = rules[rules['lift'] >= min_lift]
    return rules

### Rule Comparison and Filtering Function ###
def filter_redundant_rules(global_rules, cluster_rules):
    # Normalizing rule itemsets for accurate comparison
    global_rules['antecedents'] = global_rules['antecedents'].apply(lambda x: frozenset(x))
    global_rules['consequents'] = global_rules['consequents'].apply(lambda x: frozenset(x))
    cluster_rules['antecedents'] = cluster_rules['antecedents'].apply(lambda x: frozenset(x))
    cluster_rules['consequents'] = cluster_rules['consequents'].apply(lambda x: frozenset(x))

    # Filtering out redundant rules
    merged = cluster_rules.merge(global_rules, on=['antecedents', 'consequents'], how='left', indicator=True)
    unique_cluster_rules = merged[merged['_merge'] == 'left_only'].drop(columns='_merge')
    return unique_cluster_rules

### Main Automated Pipeline Function ###
def automated_clustering_and_rule_mining(df, eps_values, min_samples_values, min_support, min_confidence, min_lift):
    processed_df = preprocess(df)
    best_params, best_labels = estimate_dbscan_params(processed_df, eps_values, min_samples_values)
    transactions = dataframe_to_transactions(processed_df)
    initial_rules = apply_fp_growth(transactions, min_support, min_confidence, min_lift)
    
    cluster_specific_rules = {}
    for cluster_label in set(best_labels):
        if cluster_label == -1:
            continue
        cluster_transactions = dataframe_to_transactions(processed_df[best_labels == cluster_label])
        cluster_rules = apply_fp_growth(cluster_transactions, min_support, min_confidence, min_lift)
        filtered_rules = filter_redundant_rules(initial_rules, cluster_rules)
        cluster_specific_rules[cluster_label] = filtered_rules
    
    return initial_rules, cluster_specific_rules



In [16]:
# Example usage
initial_rules, cluster_specific_rules = automated_clustering_and_rule_mining(
    adult_df, 
    eps_values=np.linspace(0.1, 2.0, 20), 
    min_samples_values=range(2, 20),
    min_support=0.05, 
    min_confidence=0.2, 
    min_lift=1.5
)

# Example usage
initial_rules_bank, cluster_specific_rules_bank = automated_clustering_and_rule_mining(
    bank_df, 
    eps_values=np.linspace(0.1, 2.0, 20), 
    min_samples_values=range(2, 20),
    min_support=0.05, 
    min_confidence=0.2, 
    min_lift=1.5
)


In [17]:
initial_rules_bank

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
3331,(job_blue-collar),(education_primary),0.209246,0.149967,0.081619,0.390063,2.600998,0.050239,1.393642,0.778412
3332,(education_primary),(job_blue-collar),0.149967,0.209246,0.081619,0.544248,2.600998,0.050239,1.735053,0.724127
4052,"(job_blue-collar, default_no)",(education_primary),0.206149,0.149967,0.080734,0.391631,2.611450,0.049819,1.397233,0.777313
4054,"(default_no, education_primary)",(job_blue-collar),0.147755,0.209246,0.080734,0.546407,2.611318,0.049817,1.743313,0.724031
4055,(job_blue-collar),"(default_no, education_primary)",0.209246,0.147755,0.080734,0.385835,2.611318,0.049817,1.387649,0.780333
...,...,...,...,...,...,...,...,...,...,...
50049,"(housing_yes, loan_no, month_jul)","(poutcome_unknown, contact_cellular)",0.060385,0.478655,0.051095,0.846154,1.767773,0.022191,3.388741,0.462228
50053,"(housing_yes, month_jul)","(poutcome_unknown, loan_no, contact_cellular)",0.076311,0.399690,0.051095,0.669565,1.675210,0.020594,1.816727,0.436359
50054,"(loan_no, month_jul)","(poutcome_unknown, housing_yes, contact_cellular)",0.109710,0.217872,0.051095,0.465726,2.137611,0.027192,1.463907,0.597769
50056,(month_jul),"(poutcome_unknown, housing_yes, loan_no, conta...",0.156160,0.180933,0.051095,0.327195,1.808375,0.022840,1.217392,0.529742
